In [78]:
# 四只鸭子在一个圆形的水池里，每只鸭子的位置都是随机的。请问这四只鸭子在同一个半圆里的概率有多大？
# 通用解，包括>180度的解：通过求不可能的概率，即N个鸭子不可能在D角度的概率T。然后Ans=1-T
# 不存在一个角度D可以包括所有鸭子，就说明相邻鸭子之间的角度必须小于C=360-D
# 任意取一只鸭子作为0点，相邻鸭子间的距离为x_1，x_2，x_3，。。。x_{N-1}。
# 那么可以得到N个约束条件 0 <= x_i <= C, 360 - C <= sum(x_i) <= 360,这N个约束条件可以构成一个闭合的凸包 X
# 考虑全部的取值空间，即N只鸭子在圆上分布的所有可能性，会得到 0 <= a_i <= 360, sum(a_i) <= 360，同样这N个条件也可以构成一个闭合的凸包 A
# 不可能的概率T = X的体积/A的体积
# 为了简化，将不等式全部除以360
# A的体积很好求，是1/(n-1)!

# 问题在于如何求出X的体积
# 这时，我们把通过不等式两侧同时除C把问题标准化，x_i是服从于[0,1]均匀分布的样本，p = C/360。
# O <= x_i <= 1, (1-p)/p <= sum(x_i) <= 1/p
# sum(x_i) 服从于 Irwin-Hasll distribution, cdf(n-1,p)
# 所以X的体积是p^(n-1) * (cdf(n-1, 1/p) - cdf(n-1,(1-p)/p))

# 最终答案是 1 - p^(n-1) * (cdf(n-1, 1/p) - cdf(n-1,(1-p)/p)) / (n-1)!



import numpy as np
import math
from numpy import random
from fractions import Fraction

In [105]:
def sample(n):
    return random.rand(n)


def testOne(n,p):
    x = sample(n)
    x.sort()
    if x[0] > p or 1 - x[-1] > p:
        return False
    for i in range(1,n):
        if x[i] - x[i-1] > p:
            return False
    return True


def testT(n,p,t):
    r = 0
    for i in range(t):
        if testOne(n,p):
            r += 1
    return (r/t, r, t)

def testDuckT(n,degree,t):
    p, r, _= testT(n-1, 1 - degree / 360, t)
    return 1 - p, t - r, t

print(testDuckT(4,190,100000))

(0.58697, 58697, 100000)


In [106]:
# Irwin-Hasll distribution

def F(n):
    return math.factorial(n)

def C(n,m):
    ret = 1
    for i in range(m):
        ret *= (n - i)
    for i in range(1, m+1):
        ret //= i
    return ret

def cdf(n,x):
    int_x = int(x)
    ret = 0
    for k in range(int_x+1):
        #print(k,((-1)**k),C(n,k),((x - k)**n),((-1)**k) * C(n,k) * ((x - k)**n))
        ret += ((-1)**k) * C(n,k) * ((x - k)**n)
    return ret / F(n)

def no_sol_prob(n,p,debug=False):
    # n * p < 1
    # n < 1 / p
    a = cdf(n, 1 / p)
    if debug:
        print("a", n, 1/p, cdf(n,1/p), a)
    # n * p > 1 - p
    # n > (1 - p) / p
    b = cdf(n, (1 - p) / p)
    if debug:
        print("b", n, (1-p)/p, cdf(n,(1-p)/p), b)
        print("a-b",a-b)
    return a - b

def no_sol_prob_scaled(n, _p,debug=False):
    p = Fraction(_p)
    region_all = Fraction(1,F(n))
    if debug:
        print("region_all", region_all)
    region_p = p**n
    if debug:
        print("region_p", p, n, region_p)
        print("p/all", region_p / region_all)
    no_sol_region = no_sol_prob(n, p) * region_p
    if debug:
        print("no_sol_region", no_sol_prob(n,p,True), no_sol_region)
    return no_sol_region / region_all


def sol_prod(n, p):
    return 1 - no_sol_prob_scaled(n, p)

print(ans_prob(3, 0.5))


def duck_circle(n, degree):
    ans = sol_prod(n-1, Fraction(360-degree,360))
    print("the probability of %d ducks being within degree %d is %s, %s" % (n, degree, ans, float(ans)))
    return ans

duck_circle(4,190)
    

1/2
the probability of 4 ducks being within degree 190 is 6847/11664, 0.587019890260631


Fraction(6847, 11664)

In [74]:
1/6

0.16666666666666666